In [1]:
import sys, os
from pathlib import Path

sys.path.append(str(Path(os.path.abspath("")).parent))
import config

In [2]:
from dataset import get_data_loader, LearningTask
from transfer_learning import get_node_embeddings, get_premise_embeddings, load_model
from train import get_model

In [3]:
PREMISE_MODEL_DIR = '../' + 'experiments/thesis/graph_depth/premise/num_convolutional_layers_1/'
SIMILARITY_MODEL_DIR = '../' + 'experiments/thesis/graph_depth/similarity/num_convolutional_layers_1/'

IDS = '../' + 'id_files/validation.txt'

dataset_params = {}

In [4]:
EXPERIMENT_DIR = 'experiments/thesis/transfer/to_premise '

In [5]:
model_path = SIMILARITY_MODEL_DIR
model_task = LearningTask.SIMILARITY

In [6]:
new_task = LearningTask.PREMISE
# Can keep same dataset for both (if transforms are the same)
dataset = get_data_loader(IDS, task=new_task, batch_size=config.BATCH_SIZE, shuffle=True, in_memory=True, **dataset_params)

Dataset: TorchMemoryDataset(2465)


In [7]:
def get_transfer_model(model_path, model_task, new_task):
    
    # Load trained model
    gnn_model = load_model(model_path, model_task)

    # Get same model as previous - but for the new task
    new_model = get_model(model_path, new_task)

    # Set the embedding and gcn layers
    new_model.node_embedding = gnn_model.node_embedding
    new_model.gcn = gnn_model.gcn


    # Make these components non-trainable - only want to train the dense output layer
    for param in gnn_model.node_embedding.parameters():
        param.requires_grad = False


    for param in gnn_model.gcn.parameters():
        param.requires_grad = False
        
    return new_model

In [8]:
get_transfer_model(model_path, model_task, new_task)

GNNStack(
  (node_embedding): Embedding(15, 64)
  (gcn): GCNDirectional(
    (convs): ModuleList(
      (0): GCNConv(64, 64)
    )
    (lns): ModuleList()
  )
  (post_mp): DenseOutput(
    (lin): ModuleList(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): Linear(in_features=64, out_features=64, bias=True)
    )
    (out): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [9]:
new_model(next(iter(dataset)))

NameError: name 'new_model' is not defined